In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

C:\Users\ANIKET\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\ANIKET\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [8]:
def scrape_smartprix_for_brand(brand_slug, min_count=100):
    print(f"\n📱 Starting scrape for brand: {brand_slug}")
    url = f"https://www.smartprix.com/mobiles/{brand_slug}-brand"

    options = Options()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)

    wait = WebDriverWait(driver, 15)
    data = []
    seen_names = set()

    while len(data) < min_count:
        print(f"🔄 Loaded {len(data)} phones... Trying to load more if available.")

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        try:
            load_more_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "sm-load-more")))
            driver.execute_script("arguments[0].click();", load_more_btn)
            time.sleep(3)
        except:
            print("⚠️ Load More button not found or no more products to load.")
            break

        cards = driver.find_elements(By.CSS_SELECTOR, ".sm-product")

        for card in cards:
            try:
                name = card.find_element(By.CSS_SELECTOR, "a.name.clamp-2").text.strip()
                if name in seen_names:
                    continue
                seen_names.add(name)

                price = card.find_element(By.CSS_SELECTOR, "span.price").text.strip()

                try:
                    rating_style = card.find_element(By.CSS_SELECTOR, ".rating span.sm-rating").get_attribute("style")
                    rating = rating_style.split(":")[1].replace(";", "").strip()
                except:
                    rating = "NA"

                specs = card.find_elements(By.CSS_SELECTOR, "ul.sm-feat li")
                spec_texts = [s.text.strip() for s in specs][:6]
                while len(spec_texts) <6:
                    spec_texts.append("NA")

                data.append([name, price, rating] + spec_texts + [brand_slug])
                print(f"✅ {len(data)}: {name} | {price} | Rating: {rating}")

            except Exception as e:
                print(f"❌ Error parsing card: {e}")

    driver.quit()

    df = pd.DataFrame(data, columns=[
        "Name", "Price", "Rating", "Spec1", "Spec2", "Spec3", "Spec4", "Spec5",'Spec6', "Brand"
    ])
    
    filename = f"{brand_slug}_smartphones.csv"
    df.to_csv(filename, index=False)
    print(f"💾 Data saved to {filename}")

In [9]:
if __name__ == "__main__":
    brand_list = ["apple", "vivo", "samsung", "motorola", "realme", "oppo", "iqoo", "xiaomi", "oneplus"]
    for brand in brand_list:
        scrape_smartprix_for_brand(brand, min_count=120)



📱 Starting scrape for brand: apple
🔄 Loaded 0 phones... Trying to load more if available.
✅ 1: Apple iPhone 16 | ₹72,400 | Rating: 4.75
✅ 2: Apple iPhone 17 Air | ₹79,990 | Rating: 4
✅ 3: Apple iPhone 15 | ₹61,400 | Rating: 4.65
✅ 4: Apple iPhone 16 Pro Max | ₹1,33,900 | Rating: 4.2
✅ 5: Apple iPhone 13 | ₹43,900 | Rating: 4.35
✅ 6: Apple iPhone 17 | ₹89,990 | Rating: 4.6
✅ 7: Apple iPhone 16 Pro | ₹1,10,900 | Rating: 4.25
✅ 8: Apple iPhone 16e | ₹53,600 | Rating: 4.55
✅ 9: Apple iPhone 17 Pro Max | ₹1,69,990 | Rating: 4.25
✅ 10: Apple iPhone 17 Pro | ₹1,49,990 | Rating: 4.1
✅ 11: Apple iPhone 15 Plus | ₹72,490 | Rating: 4.75
✅ 12: Apple iPhone 14 | ₹53,890 | Rating: 4.65
✅ 13: Apple iPhone 17e | ₹64,990 | Rating: 4.7
✅ 14: Apple iPhone 16 Plus | ₹82,400 | Rating: 4.05
✅ 15: Apple iPhone Fold | ₹1,99,990 | Rating: 4.05
✅ 16: Apple iPhone 16 (256GB) | ₹83,900 | Rating: 4.4
✅ 17: Apple iPhone 15 (256GB) | ₹70,800 | Rating: 4.15
✅ 18: Apple iPhone 16 Pro (256GB) | ₹1,21,900 | Rating: 4.2

✅ 23: Vivo X Fold 5 5G | ₹1,49,999 | Rating: 4.35
✅ 24: Vivo V40e 5G | ₹22,999 | Rating: 4.35
✅ 25: Vivo V50e 5G | ₹27,999 | Rating: 4.05
✅ 26: Vivo X200 5G | ₹65,999 | Rating: 4.6
✅ 27: Vivo T3 Ultra | ₹27,998 | Rating: 4.1
✅ 28: Vivo T4 Lite 5G (6 GB RAM +128 GB) | ₹11,593 | Rating: 4.05
✅ 29: Vivo T5x 5G | ₹15,990 | Rating: 4.1
✅ 30: Vivo V50 Pro 5G | ₹54,990 | Rating: 4.2
✅ 31: Vivo Y300 5G | ₹20,999 | Rating: 4.15
✅ 32: Vivo X300 | ₹54,990 | Rating: 4.5
✅ 33: Vivo T4 5G (12GB RAM + 256GB) | ₹25,999 | Rating: 4.5
✅ 34: Vivo Y04 | ₹8,990 | Rating: 4.45
✅ 35: Vivo Y39 5G | ₹16,899 | Rating: 4.15
✅ 36: Vivo V29 Pro | ₹22,000 | Rating: 4.4
✅ 37: Vivo Y19s | ₹8,990 | Rating: 4.5
✅ 38: Vivo Y100A (8GB RAM + 256GB) | ₹13,000 | Rating: 4.1
✅ 39: Vivo V50 5G (8GB RAM + 256GB) | ₹36,999 | Rating: 4.05
✅ 40: Vivo X300 Pro 5G | ₹69,990 | Rating: 4.35
✅ 41: Realme 15 Pro 5G | ₹31,999 | Rating: NA
✅ 42: OPPO Reno 14 5G | ₹37,998 | Rating: NA
✅ 43: OPPO Reno 14 Pro 5G | ₹49,999 | Rating: NA
🔄 Loa

✅ 52: Samsung Galaxy S25 Plus | ₹82,999 | Rating: 4.35
✅ 53: Samsung Galaxy S23 FE 5G | ₹37,435 | Rating: 4.55
✅ 54: Samsung W25 | ₹1,99,990 | Rating: 4.7
✅ 55: Samsung Galaxy F06 5G | ₹9,100 | Rating: 4.05
✅ 56: Samsung Galaxy S25 Ultra (12GB RAM + 512GB) | ₹1,24,999 | Rating: 4.65
✅ 57: Samsung Galaxy A07 5G | ₹9,499 | Rating: 4.3
✅ 58: Samsung Galaxy A55 5G (8GB RAM + 256GB) | ₹29,999 | Rating: 4.6
✅ 59: Samsung Galaxy A55 5G (12GB RAM + 256GB) | ₹33,999 | Rating: 4.15
✅ 60: Samsung Galaxy S24 Ultra (12GB RAM + 512GB) | ₹1,19,999 | Rating: 4.3
✅ 61: Samsung Galaxy S25 Edge | ₹1,04,999 | Rating: 4.45
✅ 62: Samsung Galaxy F55 5G (8GB RAM + 256GB) | ₹19,877 | Rating: 4.2
🔄 Loaded 62 phones... Trying to load more if available.
✅ 63: Samsung Galaxy A74 5G | ₹42,999 | Rating: 4.55
✅ 64: Samsung Galaxy Z Fold 7 (12GB RAM + 512GB) | ₹1,74,999 | Rating: 4.75
✅ 65: Samsung Galaxy M06 (4GB RAM + 64GB) | ₹10,799 | Rating: 4.65
✅ 66: Samsung Galaxy M37 5G | ₹17,990 | Rating: 4.4
✅ 67: Samsung Ga

✅ 61: Motorola Moto G24 5G | ₹11,999 | Rating: 4.7
✅ 62: Motorola Razr 50 | ₹44,999 | Rating: 4.7
✅ 63: Motorola Moto G32 (8GB RAM + 128GB) | ₹10,999 | Rating: 4.75
✅ 64: Motorola Moto G31 | ₹8,590 | Rating: 4.35
🔄 Loaded 64 phones... Trying to load more if available.
✅ 65: Motorola Razr 70 Ultra | ₹1,09,990 | Rating: 4
✅ 66: Motorola Moto G Stylus 5G 2025 | ₹34,990 | Rating: 4.2
✅ 67: Motorola Moto G47 5G | ₹14,990 | Rating: 4.1
✅ 68: Motorola Moto A10V | ₹1,286 | Rating: 4
✅ 69: Motorola Moto A10G | ₹999 | Rating: 4.1
✅ 70: Motorola Moto G55 | ₹23,990 | Rating: 4.45
✅ 71: Motorola Moto G15 | ₹12,990 | Rating: 4.55
✅ 72: Motorola Razr 50 Ultra | ₹72,999 | Rating: 4.7
✅ 73: Motorola Rizr Rollable | ₹1,59,990 | Rating: 4.1
✅ 74: Motorola Moto G73 | ₹12,999 | Rating: 4.05
✅ 75: Motorola Moto E13 | ₹5,999 | Rating: 4.55
✅ 76: Motorola Moto G72 4G | ₹16,999 | Rating: 4.65
✅ 77: Motorola Moto Edge 40 Lite 5G | ₹19,999 | Rating: 4
✅ 78: Motorola Moto G51 5G | ₹11,999 | Rating: 4.3
✅ 79: Moto

✅ 83: Realme C73 (4GB RAM + 128GB) | ₹11,118 | Rating: 4.65
✅ 84: Realme 13 Plus 5G (8GB RAM + 256GB) | ₹19,999 | Rating: 4.65
✅ 85: Realme 13 Pro Plus 5G (12GB RAM + 512GB) | ₹27,990 | Rating: 4.25
🔄 Loaded 85 phones... Trying to load more if available.
✅ 86: Realme 13 Plus 5G | ₹15,999 | Rating: 4.15
✅ 87: Realme 12 Pro 5G | ₹19,999 | Rating: 4.4
✅ 88: Realme GT 7 (12GB RAM + 256GB) | ₹42,998 | Rating: 4.7
✅ 89: Realme GT 7T (12GB RAM + 256GB) | ₹33,997 | Rating: 4.3
✅ 90: Realme C75 5G (6GB RAM + 128GB) | ₹13,487 | Rating: 4.35
✅ 91: Realme 14x (8GB RAM + 128GB) | ₹15,196 | Rating: 4.1
✅ 92: Realme 14 Pro Lite 5G | ₹21,999 | Rating: 4.25
✅ 93: Realme P2 Pro (12GB RAM + 512GB) | ₹23,999 | Rating: 4.3
✅ 94: Realme 13 5G | ₹13,997 | Rating: 4.65
✅ 95: Realme GT 2 Pro 5G | ₹29,499 | Rating: 4.55
✅ 96: Realme Narzo 80 Turbo 5G | ₹17,990 | Rating: 4.35
✅ 97: Realme 13 Pro Plus 5G (12GB RAM + 256GB) | ₹26,999 | Rating: 4
✅ 98: Realme C61 (6GB RAM + 128GB) | ₹8,499 | Rating: 4.65
✅ 99: Real

✅ 85: Oppo K12 | ₹21,999 | Rating: 4.4
✅ 86: Oppo Reno 11F 5G | ₹25,990 | Rating: 4.2
🔄 Loaded 86 phones... Trying to load more if available.
✅ 87: OPPO A59 5G | ₹13,994 | Rating: 4.4
✅ 88: OPPO A79 5G | ₹18,449 | Rating: 4.2
✅ 89: Oppo A78 | ₹16,490 | Rating: 4.7
✅ 90: OPPO A57 4G (4GB RAM + 64 GB) | ₹10,599 | Rating: 4.25
✅ 91: Oppo Find X7 Pro 5G | ₹79,990 | Rating: 4.35
✅ 92: Oppo Find X7 5G | ₹47,999 | Rating: 4.4
✅ 93: OPPO F23 | ₹21,899 | Rating: 4.6
✅ 94: OPPO F21 Pro 4G | ₹15,999 | Rating: 4.35
✅ 95: OPPO K10 | ₹13,490 | Rating: 4.35
✅ 96: Oppo A5 4G | ₹14,990 | Rating: 4.75
✅ 97: Oppo A5x | ₹12,990 | Rating: 4
✅ 98: Oppo Find X8s Plus | ₹49,990 | Rating: 4.1
✅ 99: Oppo Tri Fold Phone | ₹1,99,990 | Rating: 4.65
✅ 100: Oppo A1s 5G | ₹13,990 | Rating: 4.75
✅ 101: OPPO A59 5G (6GB RAM + 128GB) | ₹15,494 | Rating: 4.5
✅ 102: OPPO A38 | ₹11,590 | Rating: 4.3
✅ 103: OPPO A58 4G | ₹12,890 | Rating: 4.2
✅ 104: Oppo A78 4G | ₹16,498 | Rating: 4.25
✅ 105: OPPO F21s Pro 4G | ₹17,706 | Ra

✅ 120: iQOO Neo 5 Life | ₹22,999 | Rating: 4.1
✅ 121: iQOO 7 (8GB RAM + 256GB) | ₹25,499 | Rating: 4.05
✅ 122: iQOO 7 Legend (12GB RAM + 256GB) | ₹43,990 | Rating: 4.6
✅ 123: iQOO Z3 | ₹19,990 | Rating: 4.25
✅ 124: iQOO 7 (12GB RAM + 256GB) | ₹26,499 | Rating: 4.5
✅ 125: iQOO 6 | ₹43,999 | Rating: 4.45
✅ 126: iQOO 3 Pro | ₹39,990 | Rating: 4.2
💾 Data saved to iqoo_smartphones.csv

📱 Starting scrape for brand: xiaomi
🔄 Loaded 0 phones... Trying to load more if available.
✅ 1: Xiaomi Redmi Note 14 SE 5G | ₹14,999 | Rating: 4.6
✅ 2: Xiaomi Redmi Turbo 4 Pro | ₹23,990 | Rating: 4.2
✅ 3: Xiaomi Redmi 15 5G | ₹17,999 | Rating: 4.55
✅ 4: Xiaomi Redmi Note 14 Pro Plus | ₹25,700 | Rating: 4.4
✅ 5: Xiaomi Redmi Note 14 | ₹16,998 | Rating: 4.35
✅ 6: Xiaomi 14 Civi | ₹29,999 | Rating: 4.7
✅ 7: Xiaomi Redmi 13 5G | ₹11,749 | Rating: 4.5
✅ 8: Xiaomi Redmi Note 15 Pro Plus | ₹35,990 | Rating: 4.65
✅ 9: Xiaomi Redmi A4 5G | ₹8,499 | Rating: 4.6
✅ 10: Xiaomi Redmi Note 14 (8GB RAM + 256GB) | ₹19,998 | 

✅ 15: OnePlus Nord 4 5G | ₹27,990 | Rating: 4.35
✅ 16: OnePlus 15 | ₹79,999 | Rating: 4.5
✅ 17: OnePlus Ace 6 Ultra | ₹39,990 | Rating: 4.75
✅ 18: OnePlus 13R (16GB RAM + 512GB) | ₹47,997 | Rating: 4.55
✅ 19: Oneplus 14R | ₹49,990 | Rating: 4.05
✅ 20: OnePlus Ace 6 | ₹29,990 | Rating: 4.5
✅ 21: OnePlus 13 (24GB RAM+ 1TB) | ₹89,997 | Rating: 4.35
✅ 22: OnePlus 13 Pro | ₹70,990 | Rating: 4.15
✅ 23: OnePlus 13s (12GB RAM + 512GB) | ₹59,998 | Rating: 4.4
✅ 24: OnePlus Nord CE 5 Lite 5G | ₹22,990 | Rating: 4.05
✅ 25: OnePlus Nord CE 4 Lite 5G (8GB RAM + 256GB) | ₹20,996 | Rating: 4.5
✅ 26: Oneplus 14 Pro | ₹89,990 | Rating: 4.15
✅ 27: OnePlus 13 (16GB RAM+ 512GB) | ₹76,997 | Rating: 4.25
✅ 28: OnePlus 11R 5G | ₹36,899 | Rating: 4.4
✅ 29: OnePlus Nord 3 5G | ₹22,994 | Rating: 4.15
✅ 30: OnePlus 12R (8GB RAM + 256GB) | ₹37,978 | Rating: 4.55
✅ 31: OnePlus Nord 6 | ₹34,999 | Rating: 4.6
✅ 32: OnePlus 13 Mini | ₹49,990 | Rating: 4.55
✅ 33: OnePlus 10 Pro 5G | ₹34,990 | Rating: 4.65
✅ 34: OnePlu